# Use network isolation with managed online endpoints

In [ ]:
# %pip install azure-mgmt-resource
# %pip install azure-mgmt-compute
# %pip install azure-mgmt-network
# %pip install azure-identity
%pip install azure-mgmt-containerregistry

## Connect to Azure Machine Learning

### Import Required Libraries

In [1]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.mgmt.resource import ResourceManagementClient, DeploymentScriptsClient
from azure.mgmt.resource.templatespecs import TemplateSpecsClient
from azure.mgmt.resource.templatespecs.models import TemplateSpec
from azure.mgmt.resource.resources.v2021_04_01 import models as resource_models
from azure.mgmt.resource.templatespecs.v2022_02_01.models import TemplateSpec
from azure.mgmt.network import NetworkManagementClient
from azure.mgmt.compute import ComputeManagementClient
from azure.mgmt.compute.models import VirtualMachineRunCommand, VirtualMachine, VirtualMachineRunCommandScriptSource,RunCommandInputParameter
from azure.identity import DefaultAzureCredential, AzureCliCredential
from azure.mgmt.deploymentmanager import AzureDeploymentManager
from azure.mgmt.containerregistry import ContainerRegistryManagementClient
from azure.mgmt.containerregistry.models import EncodedTaskRunRequest
import os, json, random
from pathlib import Path

### Set variables

In [8]:
_rand = random.randint(0,10000)
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace_name = "<AML_WORKSPACE_NAME>"
location = "<LOCATION>"
suffix = f"netiso{_rand}"

In [9]:
subscription_id = "6fe1c377-b645-4e8e-b588-52e57cc856b2"
resource_group = "alwvnettest"
workspace = "alwallace"

In [10]:
resource_group="alwallace-rg"
suffix="testfoo"
location = "eastus2"

In [11]:
vnet_name = f"vnet-{suffix}"
subnet_name = "snet-scoring"
endpoint_name = f"endpoint-{suffix}"
vm_name="moevnet-vm"
identity_name = f"uai{suffix}"

## Get clients

In [13]:
credential = DefaultAzureCredential()

ml_client = MLClient(credential, subscription_id, resource_group, workspace)

network_client = NetworkManagementClient(credential, subscription_id)

resource_client = ResourceManagementClient(credential, subscription_id)

compute_client = ComputeManagementClient(credential, subscription_id)

ts_client = TemplateSpecsClient(credential, subscription_id)

cr_client = ContainerRegistryManagementClient(credential, subscription_id)

In [15]:
m=list(ml_client.models.list())[0]

Model({'job_name': None, 'is_anonymous': False, 'auto_increment_version': True, 'name': 'test1', 'description': None, 'tags': {}, 'properties': {}, 'id': '/subscriptions/6fe1c377-b645-4e8e-b588-52e57cc856b2/resourceGroups/alwallace-rg/providers/Microsoft.MachineLearningServices/workspaces/alwallace/models/test1', 'Resource__source_path': None, 'base_path': '/mnt/samples/azureml-examples.worktrees/alwallace/networkisolation/sdk/python/endpoints/online/managed', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f196e76fb50>, 'serialize': <msrest.serialization.Serializer object at 0x7f1974aba800>, 'version': None, 'latest_version': '1', 'path': None, 'datastore': None, 'utc_time_created': None, 'flavors': None, 'arm_type': 'model_version', 'type': 'custom_model'})

## Create Resources

### Create identity

In [ ]:
deployment = resource_client.deployments.begin_create_or_update(
    resource_group_name=resource_group,
    deployment_name=f"{suffix}-uai",
    parameters={"properties" : {
        "mode" : "Incremental",
        "template" : json.loads(Path("vnet/setup_ws/uai.json").read_text()), 
        "parameters" : {
            "suffix" : {
                "value" : f"{suffix}"},
        }
        }
    }
).result()

### Create workspace

In [ ]:
tempspec_root = Path("vnet/setup_ws")

tempspec = ts_client.template_spec_versions.create_or_update(
    template_spec_name="test",
    template_spec_version=str(_rand),
    resource_group_name=resource_group,
    template_spec_version_model={
        "location": "eastus",
        "linked_templates": [
            {
                "template": json.loads(f.read_text()),
                "path": str(f.relative_to(tempspec_root)),
            }
            for f in (tempspec_root / "modules").iterdir()
        ],
        "main_template": json.loads((tempspec_root / "main-linked.json").read_text()),
    }
)

In [ ]:
deployment = resource_client.deployments.begin_create_or_update(
    resource_group_name=resource_group,
    deployment_name=suffix,
    parameters={ "properties" :{
            "mode" : "Incremental",
            "template_link": {
                "id" : tempspec.id
            },
            "parameters" : {
                "suffix": {
                    "value" : suffix
                }
            }
        }
    }
).result()

### Create vm

In [ ]:
deployment = resource_client.deployments.begin_create_or_update(
    resource_group_name=resource_group,
    deployment_name=f"{suffix}-vm",
    parameters={"properties" : {
        "mode" : "Incremental",
        "template" : json.loads(Path("vnet/setup_vm/vm-main.json").read_text()), 
        "parameters" : {
            "vnetName" : {
                "value" : f"vnet-{suffix}"
                }
            }
        }
    }
).result()

## Continue on the VM

### Execute the setup script

In [34]:
command = compute_client.virtual_machine_run_commands.begin_create_or_update(
    vm_name=vm_name,
    resource_group_name=resource_group,
    run_command_name="test",
    run_command={"source": 
        {"script": Path("vnet/setup_vm/scripts/vmsetup.sh").read_text()},
        "parameters": [
            {"name" : "SUBSCRIPTION",
            "value" : subscription_id},
            {"name" : "RESOURCE_GROUP",
            "value" : resource_group},
            {"name" : "LOCATION",
            "value" : location},
            {"name" : "WORKSPACE",
            "value" : workspace},
            {"name" : "IDENTITY_NAME",
            "value" : identity_name}
        ],
        "location": location} 
).result()

In [ ]:
l=compute_client.virtual_machine_run_commands.list_by_virtual_machine(resource_group_name=resource_group, vm_name=vm_name)

In [ ]:
cr_client.build_tasks.begin_create(
    type
    registry_name="alwallace",
    resource_group_name=resource_group,
    task_run_request=EncodedTaskRunRequest(),
)
